# Session 04: Model Context Protocol (MCP)

MCP (Model Context Protocol; https://modelcontextprotocol.io/docs/getting-started/intro) is an open-source standard for connecting LLMs with external systems.
It is commonly implemented as a REST server that exposes capabilities in a predefined protocol.

MCP servers can provide three main types of capabilities:

| Feature	 | Explanation	                                                                                                                                                                                                  | Examples	                |
|:-----------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------|
| Resources	 | Passive data sources that provide read-only access to information for context.  Similar to GET endpoints in a REST API - they provide data but shouldn't perform significant computation or have side effects | "Access knowledge base"	 |
| Tools	     | Functions that the LLM can actively call, and decides when to use them based on user requests. Tools are expected to perform computation and can have side effects.                                           | "Book a flight"	         |
| Prompts    | Pre-built instruction templates that tell the model to work with specific tools and resources.                                                                                                                | "Summarize my meeting"   |


In [ ]:
import json
from IPython.display import display, Markdown
from typing import Literal

def mdprint(text):
    """Helper function for printing markdown text."""
    display(Markdown(text))

def pprint(result):
    """Helper function for pretty-printing raw model responses."""
    for item in result.new_items:
        print(item.__class__, json.dumps(item.to_input_item(), indent=2))

## Instantiating the Server

Use the official MCP Python SDK (`mcp`; https://github.com/modelcontextprotocol/python-sdk)  to quickly instantiate a server:

In [ ]:
from mcp.server.fastmcp import FastMCP

# Create server
server = FastMCP("Weather Recommendation Bot")

## Adding Tools

Add three tools to the server:
- a tool to get the current time
- a tool to get the current location
- a tool to get the weather forecast for a given location

Adding tools is similar to how it is done in `agents`: just annotate your tool call function; use the `@server.tool()` annotation to register the tool to you `server` object.

## Adding resources

Similarly, we can add the `@server.resource(<uri>)` decorate to register resources. Implement a resource that returns the users clothing inventory (use mock data).

*Note*: this might not be supported for all models and APIs; if you face issues with models using it, register it as a tool instead.

In [ ]:
mock_data = [
    ("jackets", "wool jacket", "charcoal gray", 5),
    ("jackets", "rain jacket", "navy blue", 4),
    ("jackets", "winter coat", "forest green", 4),
    ("jackets", "blazer", "midnight blue", 2),
    ("jackets", "denim jacket", "faded blue", 3),
    ("jackets", "windbreaker", "bright red", 4),
    ("jackets", "leather jacket", "black", 3),
    ("jackets", "puffer jacket", "olive green", 5),

    ("upper_layers", "t-shirt (short sleeve)", "white", 5),
    ("upper_layers", "t-shirt (long sleeve)", "heather gray", 4),
    ("upper_layers", "polo shirt", "cobalt blue", 3),
    ("upper_layers", "flannel shirt", "red plaid", 4),
    ("upper_layers", "dress shirt", "light blue", 2),
    ("upper_layers", "hoodie (lightweight)", "charcoal", 5),
    ("upper_layers", "hoodie (heavyweight)", "dark green", 4),
    ("upper_layers", "sweater (thin)", "cream", 4),
    ("upper_layers", "sweater (thick wool)", "oatmeal", 5),
    ("upper_layers", "cardigan", "burgundy", 3),
    ("upper_layers", "thermal base layer", "black", 4),

    ("lower_layers", "jeans (regular)", "indigo", 5),
    ("lower_layers", "jeans (thermal lined)", "dark wash", 4),
    ("lower_layers", "chinos", "khaki", 3),
    ("lower_layers", "cargo pants", "olive drab", 4),
    ("lower_layers", "shorts", "light gray", 5),
    ("lower_layers", "sweatpants", "charcoal", 4),
    ("lower_layers", "dress pants", "navy", 2),
    ("lower_layers", "leggings/base layer", "black", 3),

    ("footwear", "sneakers", "white", 5),
    ("footwear", "running shoes", "neon green", 4),
    ("footwear", "boots (waterproof)", "brown", 5),
    ("footwear", "boots (winter insulated)", "black", 4),
    ("footwear", "sandals", "tan", 3),
    ("footwear", "dress shoes", "oxblood", 2),
    ("footwear", "hiking boots", "dark brown", 4),

    ("accessories", "umbrella", "black", 5),
    ("accessories", "sunglasses", "matte black", 4),
    ("accessories", "baseball cap", "navy", 4),
    ("accessories", "beanie", "charcoal", 5),
    ("accessories", "sun hat (wide brim)", "beige", 3),
    ("accessories", "scarf (light)", "sky blue", 3),
    ("accessories", "scarf (wool)", "dark gray", 4),
    ("accessories", "gloves (light)", "black", 3),
    ("accessories", "gloves (winter insulated)", "navy", 4),
    ("accessories", "mittens", "red", 3),
    ("accessories", "neck gaiter", "dark green", 4),

    ("specialized", "rain pants", "black", 3),
    ("specialized", "snow pants", "charcoal", 4),
    ("specialized", "vest (puffer)", "navy", 4),
    ("specialized", "vest (fleece)", "gray", 3),
    ("specialized", "rain poncho", "yellow", 2),
    ("specialized", "ear muffs", "black", 3),
    ("specialized", "face mask (cold weather)", "dark gray", 4),
]


## Adding Prompts

Prompts are essentially server-side functions to transform structure *input*  into a natural language prompt for the model. These are not called by the model directly, but instead by the user to obtain model input.

You can register a prompt template with `@server.prompt(title=...)`.

In [ ]:
from agents import Agent, Runner, OpenAIChatCompletionsModel
from agents.mcp import MCPServerStreamableHttp

from openai import AsyncOpenAI
from api_key import API_KEY
API_URL = "https://api.helmholtz-blablador.fz-juelich.de/v1/"
#API_KEY = "<KEY>"
API_MODEL = "1 - GPT-OSS-120b - an open model released by OpenAI in August 2025"

In [ ]:
async def run(...]) -> str:
    async with MCPServerStreamableHttp(params={"url": "http://127.0.0.1:8000/mcp"}) as server:
        agent = Agent(
            name="Agent",
            mcp_servers=[server],
            model=OpenAIChatCompletionsModel(
                model=API_MODEL,
                openai_client=AsyncOpenAI(api_key=API_KEY, base_url=API_URL)
            )
        )
        prompt = ...
        out = ...
    return out

In [ ]:
out = await run(style="sporty", time="tomorrow")
mdprint(out.final_output)

In [ ]:
out = await run(style="elegant", time="today")
mdprint(out.final_output)